In [80]:
import numpy as np
import pickle
import sys, os
sys.path.insert(0, os.path.abspath(".."))
import matplotlib.pyplot as plt
np.set_printoptions(precision=5, suppress=True, linewidth=100)
plt.rcParams['figure.dpi'] = 150
import tenpy
tenpy.tools.misc.setup_logging(to_stdout="INFO")
from src.MPO_functions import MPS_drop_charge, double_to_mpo, mpo_product, mpo_trace, truncate_mpo
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
torch.set_default_dtype(torch.float64)
import random



### Calculate Renyi-m correlators for m=2,4,6,8 from ground state data

In [5]:
L=8
with open("..\\data\\groundstates_Z2_L"+str(L)+".pkl", "rb") as f:
    states = pickle.load(f)

In [8]:
L=8 #system size

R2=[]
R4=[]
R6=[]
R8=[]
for i,psi in enumerate(states):
    psi=states[i].copy()
    phi=psi.copy()
    phi.apply_local_term([('Sigmax',int((L/4-1)*2)),('Sigmax',int((L/4-1)*2+1)),('Sigmax',int(L-L/4-1)*2),('Sigmax',int(L-L/4-1)*2+1)])
    rho=double_to_mpo(MPS_drop_charge(psi),L)
    sig=double_to_mpo(MPS_drop_charge(phi),L)
    rho2=mpo_product(rho,rho,chi_max=40,eps=10e-20)
    tau=mpo_product(rho,sig,chi_max=40,eps=10e-20)
    tau2=mpo_product(tau,tau,chi_max=40,eps=10e-25)
    rho4=mpo_product(rho2,rho2,chi_max=40,eps=10e-25)
    R2.append(mpo_trace(tau)/mpo_trace(rho2))
    print(R2[i])
    tau3=mpo_product(tau2,tau,chi_max=40,eps=10e-25)
    rho6=mpo_product(rho4,rho2,chi_max=40,eps=10e-25)
    R4.append(mpo_trace(tau2)/mpo_trace(rho4))
    print(R4[i])
    R6.append(mpo_trace(tau3)/mpo_trace(rho6))
    print(R6[i])
    gc.collect()
    tau4=mpo_product(tau2,tau2,chi_max=40,eps=10e-30)
    rho8=mpo_product(rho4,rho4,chi_max=40,eps=10e-30)
    R8.append(mpo_trace(tau4)/mpo_trace(rho8))
    print(R8[i])
    print(i)

0.9999996093742739
0.999998437495879
0.9999964843648129
0.9999937499835185
0
0.9971196007153456
0.988367804296063
0.9736914228619121
0.9531910875069985
1
0.988027805974277
0.9503800622071364
0.886538679745139
0.7985986756636244
2
0.9706733256285679
0.8733922866029972
0.7097776507871467
0.512000563326728
3
0.9401382818569178
0.7319797818747972
0.4386315980871694
0.21585061233731456
4
0.8857807692605153
0.5122859125934166
0.19113011643763114
0.063660330907945
5
0.7910287325435014
0.2779269535982813
0.06238538940319851
0.014566562665439372
6
0.6486918518772847
0.12108440198144152
0.01700916220474426
0.002723833764337117
7
0.48492161861941646
0.04769975112385166
0.004296342816391706
0.00045541963544054777
8
0.34115940241822595
0.018850165451298702
0.0011054164687673194
7.675165960893069e-05
9
0.2355134405658366
0.00787272724279988
0.00030718732027346573
1.4165499142624518e-05
10
0.16380185338838904
0.003526175719253671
9.404809627502595e-05
2.954105725519494e-06
11
0.11607458529751476
0.00

In [ ]:

#write correlators to file
field_names=["L","J","U","s"]
file_name=r"..\\data\\correlators_Z2\\Ising_DMRG_L"+str(L)+".csv"

data = {
"R2": R2,
"R4": R4,
"R6": R6,
"R8": R8,
"p": ps,
}

df = pd.DataFrame(data)
df.to_csv(file_name, mode='a', index=False, header=True)

### Train neural network to learn fidelities

In [16]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42) 

In [75]:
import pandas as pd
import numpy as np
import os

def load_ising(L):
    """Load one Ising CSV file and return (X, Y)."""
    path = os.path.join("..", "data", "correlators_Z2", f"Ising_exact_L{L}.csv")
    df = pd.read_csv(path)

    Y = df["fid"].to_numpy()
    X = df[["R2", "R4", "R6", "R8"]].to_numpy()
    X = np.c_[X, L * np.ones(X.shape[0]), np.linspace(0, 2, X.shape[0])]

    return X, Y

sizes = [4, 6, 8, 10, 12, 14,16,18]  
data = {L: load_ising(L) for L in sizes}





In [78]:
X4, Y4 = data[4]
X6, Y6 = data[6]
X8, Y8 = data[8]
X10, Y10 = data[10]
X12, Y12 = data[12]
X14, Y14 = data[14]
X16, Y16 = data[16]
X18, Y18 = data[18]

In [47]:
#generates analytically known data. at p=\infty we now that all correlators vanish and at p=0, all correlators are equal to 1
physicsX_p0 = np.ones((64,6))
physicsX_p0[:,4]=np.arange(1,65)
physicsX_p0[:,5]=0
physicsY_p0=np.ones(64)

physicsX_pinf = np.zeros((64,6))
physicsX_pinf[:,4]=np.arange(1,65)
physicsX_pinf[:,5]=1e6
physicsY_pinf=np.zeros(64)

In [ ]:
X=np.concatenate((X4,X6,X8,X10,X12,X14,X16,X18,physicsX_p0,physicsX_pinf))
Y=np.concatenate((Y4,Y6,Y8,Y10,Y12,Y14,Y16,Y18,physicsY_p0,physicsY_pinf))

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [49]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float64)
y_train_tensor = torch.tensor(y_train, dtype=torch.float64).view(-1, 1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
y_test_tensor = torch.tensor(y_test, dtype=torch.float64).view(-1, 1)


In [50]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(6, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)




### Import DMRG data

In [52]:
DMRG32=pd.read_csv(r"..\\data\\correlators_Z2\\Ising_DMRG_L32.csv")
DMRG32_X=DMRG32[["R2","R4","R6","R8"]].to_numpy()
DMRG32_X=np.c_[DMRG32_X,32*np.ones(DMRG32_X.shape[0]),DMRG32["p"].to_numpy()/8]

In [54]:
DMRG64=pd.read_csv(r"..\\data\\correlators_Z2\\Ising_DMRG_L64.csv")
DMRG64_X=DMRG64[["R2","R4","R6","R8"]].to_numpy()
DMRG64_X=np.c_[DMRG64_X,64*np.ones(DMRG64_X.shape[0]),DMRG64["p"].to_numpy()/8]

In [55]:
X_prediction_tensor_32 = torch.tensor(DMRG32_X, dtype=torch.float64)
X_prediction_tensor_64 = torch.tensor(DMRG64_X, dtype=torch.float64)

In [ ]:
N_models=1000
data_32=np.zeros((len(DMRG64["p"]),N_models))
data_64=np.zeros((len(DMRG64["p"]),N_models))
training_fail=[]
for i in range(N_models):
    print(i)
    model = Net()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.002)
   


    epochs = 1000
    for epoch in range(epochs):
        model.train()
        outputs = model(X_train_tensor)
        alpha = 0
        loss_mse = criterion(outputs, y_train_tensor)
        loss=loss_mse
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():}")
    if loss.item()>1e-3:
        training_fail.append(i)
            
    max_abs_weight = max(param.abs().max().item() for param in model.parameters() if param.requires_grad)

    print("Maximum absolute weight:", max_abs_weight)

    model.eval()
    with torch.no_grad():
        results32 = model(X_prediction_tensor_32)
        results64 = model(X_prediction_tensor_64)
        data_32[:,i]=results32[:,0]
        data_64[:,i]=results64[:,0]
    print()
    print("---------- finished run ----------")
results32=data_32.mean(axis=1)
results64=data_64.mean(axis=1)

In [ ]:
file_name="..\\data\\correlators_Z2\\Fidelity_L32_L64.csv"
data = {
"Fidelity_L32": results32[:,0],
"Fidelity_L64": results64[:,0],
"p": DMRG32["p"]
}

df = pd.DataFrame(data)
